# 深度卷积神经网络

* 卷积神经网络
* 在更大、更真实的数据集上训练卷积神经网络的性能和可行性还有待研究

**classical pipelines**

1. Preprocess the dataset with **hand-crafted features** based on some knowledge of optics, geometry, other analytic tools, and occasionally on the serendipitous discoveries of lucky graduate students.
1. Feed the data through a standard set of **feature extractors** such as the SIFT (scale-invariant feature transform) , the SURF (speeded up robust features), or any number of other hand-tuned pipelines.

**计算机视觉研究人员**

* 推动领域进步的是数据特征，而不是学习算法
* 从对最终模型精度的影响来说，更大或更干净的数据集、或是稍微改进的特征提取，比任何学习算法带来的进步要大得多


## 学习表征

* the most important part of the pipeline was the representation
* 在2012年前，图像特征都是机械地计算出来的
* 事实上，设计一套新的特征函数、改进结果，并撰写论文是盛极一时的潮流

## 学习表征

* 特征本身应该被学习
* the features ought to be hierarchically composed with multiple jointly learned layers, each with learnable parameters
* In the case of an image, the lowest layers might come to detect edges, colors, and textures.

### 数据

* 2009年，ImageNet数据集发布，并发起ImageNet挑战赛：要求研究人员从100万个样本中训练模型，以区分1000个不同类别的对象
* ImageNet Challenge
    * which models performed best at a greater scale than academics had previously considered

### 硬件

* CPU
    * CPU的每个核心都拥有高时钟频率的运行能力，和高达数MB的三级缓存（L3Cache）
    * 通用核心的制造成本非常高
* GPU
    * 由$100 \sim 1000$个小的处理单元组成
    * 每个GPU核心都相对较弱，有时甚至以低于1GHz的时钟频率运行

### 硬件

* GPU
    * 但庞大的核心数量使GPU比CPU快几个数量级；
        * power consumption tends to grow *quadratically* with clock frequency
        * for the power budget of a CPU core that runs 4 times faster (a typical number),
you can use 16 GPU cores at $1/4$ the speed,
which yields $16 \times 1/4 = 4$ times the performance
    * GPU cores are much simpler
(in fact, for a long time they were not even *able*
to execute general-purpose code),
which makes them more energy efficient

### 硬件

* 卷积神经网络中的计算瓶颈：卷积和矩阵乘法，都是可以在硬件上并行化的操作
* AlexNet：使用两个显存为3GB的NVIDIA GTX580 GPU实现了快速卷积运算

## AlexNet

* 2012年，AlexNet横空出世
    * 它首次证明了学习到的特征可以超越手工设计的特征
* 一举打破了计算机视觉研究的现状
* AlexNet使用了8层卷积神经网络，并以很大的优势赢得了2012年ImageNet图像识别挑战赛

## AlexNet

[深度卷积神经网络（AlexNet）](https://www.bilibili.com/video/BV1h54y1L7oe?p=1)

仅AlexNet和之前人工特征提取的对比

## AlexNet

<center><img src="../img/alexnet.svg" width="30%"></center>
<center>从LeNet（左）到AlexNet（右）</center>

## AlexNet

* AlexNet和LeNet的设计理念非常相似，但也存在显著差异
* AlexNet比相对较小的LeNet5要深得多
* 在AlexNet的第一层，卷积窗口的形状是$11\times11$（更大的卷积窗口）
* AlexNet的卷积通道数目是LeNet的10倍
* 在最后一个卷积层后有两个全连接层，分别有4096个输出
    * 这两个巨大的全连接层拥有将近1GB的模型参数

### 激活函数
* ReLU激活函数的计算更简单
* 如果模型参数没有正确初始化
    * sigmoid函数可能在正区间内得到几乎为0的梯度
    * 从而使模型无法得到有效的训练

### Capacity Control and Preprocessing

* AlexNet通过暂退法控制全连接层的模型复杂度，而LeNet只使用了权重衰减
* AlexNet在训练时增加了大量的图像增强数据，如翻转、裁切和变色
    * 这使得模型更健壮，更大的样本量有效地减少了过拟合

In [1]:
import torch
from d2l import torch as d2l
from torch import nn

net = nn.Sequential(
    # 这里，我们使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，池化层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
    nn.Linear(6400, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
    nn.Linear(4096, 10),
)

构造一个单通道数据，来观察每一层输出的形状

$n_h$：高度；$p_h$：填充；$s_h$：步幅

$$
\lfloor(n_h-k_h+p_h+s_h)/s_h\rfloor
$$

In [2]:
X = torch.randn(1, 1, 224, 224)
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, "output shape:\t", X.shape)

Conv2d output shape:	 torch.Size([1, 96, 54, 54])
ReLU output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Conv2d output shape:	 torch.Size([1, 256, 26, 26])
ReLU output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 384, 12, 12])
ReLU output shape:	 torch.Size([1, 384, 12, 12])
Conv2d output shape:	 torch.Size([1, 256, 12, 12])
ReLU output shape:	 torch.Size([1, 256, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 256, 5, 5])
Flatten output shape:	 torch.Size([1, 6400])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1, 4096])
ReLU output shape:	 torch.Size([1, 4096])
Dropout output shape:	 torch.Size([1, 4096])
Linear output shape:	 torch.Size([1,

## 总结

* AlexNet的架构与LeNet相似，但使用了更多的卷积层和更多的参数来拟合大规模的ImageNet数据集
* AlexNet是从浅层网络到深层网络的关键一步
* 尽管AlexNet的代码只比LeNet多出几行，但学术界花了很多年才接受深度学习这一概念